In [2]:
%%sh
python convert.py

processing /home/jovyan/work/jsons/20_04_05__03_00_20.json
56256 records read from json
+-------+--------------------+------+------+---------+------------+----------------+--------+---+----------+-----+--------------------+--------------------+
|   shop|                name|weight| price|old_price|price_per_kg|old_price_per_kg|discount|cat|product_id| type|              datetm|                 url|
+-------+--------------------+------+------+---------+------------+----------------+--------+---+----------+-----+--------------------+--------------------+
|karusel| Антиперспирант-к...|    49|199.99|     null|     4081.43|            null|    null| 15|   1007746|piece|2020-04-05 03:25:...|/products/35828-a...|
|  metro| Антиперспирант-к...|    49| 190.0|     null|     3877.55|            null|    null| 15|    137598|piece|2020-04-05 03:03:...|/products/35828-a...|
|karusel| Бальзам для особ...|   225| 97.99|     null|      435.51|            null|    null| 15|   1019125|piece|2020-04-05 03

20/04/05 08:36:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
20/04/05 08:36:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [5]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [6]:
from pyspark.sql.dataframe import DataFrame
import plotly.graph_objects as go

DataFrame.old_show = getattr(DataFrame, 'old_show', DataFrame.show)
def show(self, n=20, truncate=20):
    print(self.count())
    self.old_show(n, truncate=truncate)
    return self

DataFrame.show = show

import random
def sql(self, q):
    name = 'table_{}'.format(random.randint(0000, 9999))
    self.createOrReplaceTempView(name)
    q = q.format(T=name)
    print(q)
    return df.sql_ctx.sql(q)
DataFrame.sql = sql

def plot(df, title, xaxis_title='Дата', yaxis_title='Количество товаров'):
    fig = go.Figure()
    for row in df.collect():
        name = row[0]
        first = row[1]
        fig.add_trace(go.Scatter(x=data.columns[1:], y=[i/first for i in row[1:]],
                            mode='lines+markers',
                            name=name))

    fig.update_layout(title=title,
                      xaxis_title=xaxis_title,
                      xaxis_tickmode='linear',
                      yaxis_title=yaxis_title)
    fig.show()

DataFrame.plot = plot

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType, DateType
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()

df = spark.read.parquet('store').withColumn('date', F.col('datetm').cast(DateType())).show()



826431
+-----+--------------------+------+------+---------+------------+----------------+--------+---+----------+------+--------------------+--------------------+----------+
| shop|                name|weight| price|old_price|price_per_kg|old_price_per_kg|discount|cat|product_id|  type|              datetm|                 url|      date|
+-----+--------------------+------+------+---------+------------+----------------+--------+---+----------+------+--------------------+--------------------+----------+
|metro|Проростки Микс Кр...|    70|169.89|     null|      2427.0|            null|    null|  1|    123000| piece|2020-03-16 19:36:...|/products/30784-p...|2020-03-16|
|metro|Чай черный байхов...|   100| 799.0|     null|      7990.0|            null|    null| 12|    123492| piece|2020-03-16 19:42:...|/products/16059-c...|2020-03-16|
|metro|Оливки с перцем T...|   300|  85.0|     null|      283.33|            null|    null|  7|    124369| piece|2020-03-16 20:09:...|/products/17355-o...|202

In [8]:
df.groupBy('date').count().sort(F.desc('count')).show()

15
+----------+-----+
|      date|count|
+----------+-----+
|2020-03-17|55649|
|2020-03-25|55569|
|2020-03-26|55543|
|2020-03-16|55485|
|2020-04-02|55449|
|2020-04-05|55375|
|2020-03-22|55339|
|2020-03-21|55251|
|2020-04-01|55246|
|2020-03-19|55091|
|2020-03-24|55068|
|2020-03-23|55000|
|2020-03-31|54505|
|2020-03-18|54192|
|2020-03-30|53669|
+----------+-----+



DataFrame[date: date, count: bigint]

In [9]:
df.groupBy("shop").pivot("date").agg(F.count('product_id')).persist() \
    .show().plot(title='Total products per shop', xaxis_title='Count')


4
+-------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|   shop|2020-03-16|2020-03-17|2020-03-18|2020-03-19|2020-03-21|2020-03-22|2020-03-23|2020-03-24|2020-03-25|2020-03-26|2020-03-30|2020-03-31|2020-04-01|2020-04-02|2020-04-05|
+-------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| prisma|      8991|      8689|      8629|      8660|      8680|      9008|      8929|      8919|      9045|      8946|      8588|      8927|      8997|      8883|      9017|
|  metro|     19599|     19645|     19365|     19702|     19727|     19610|     19553|     19540|     19686|     19783|     19004|     19287|     19718|     19843|     19700|
|karusel|     12501|     13139|     12450|     12642|     12803|     12711|     12459|     12557|     12756|     12656|    

NameError: name 'data' is not defined

In [ ]:
df.where('old_price is not null').groupBy("shop").pivot("date").agg(F.count('product_id')).persist() \
    .show().plot(title='Total products with discounts ', xaxis_title='Count')

In [ ]:
df.groupBy("shop").pivot("date").agg(F.sum('price')/F.count('product_id')).cache() \
    .show().plot(title='Average price per shop', xaxis_title='Count')